# Démonstration de ASLTAM

Ce jupyter notebook est destiné à l'illustration de la présentation orale du package le 13 décembre 2021. Le code est tiré du fichier ``script.py`` disponible sur le git.

### On commence par installer tout ce dont on a besoin

In [ ]:
pip install -r ../requirements.txt


In [ ]:
pip install -i https://test.pypi.org/simple/ asltam==1.1.0


### Importation du package et des données

In [ ]:
import asltam as atm

prix = atm.load_price().save_as_price(index=' ')
dist = atm.load_dist().save_as_dist(index=' Nom gare ')
geo = atm.load_geo().save_as_geo(index=' Nom gare ')
geo = geo[['Latt', 'Long']]

## Construction de la carte intéractive

In [ ]:
from ipywidgets import interact, fixed

interact(atm.trajet, DEPART=list(geo.index), ARRIVEE=list(geo.index),
     data_geo=fixed(geo), data_price=fixed(prix), data_dist=fixed(dist), 
     KEY='5b3ce3597851110001cf62486f5564a064e34f3895221e5a0d9a2405')

## Affichage des distributions avec ``interact``

In [ ]:
from ipywidgets import interact, fixed

interact(atm.kde_gare, all = True, data_dist = fixed(dist),
         data_price = fixed(prix), start = prix.columns, 
         target = prix.columns, bw = (0.05,2,0.01))

## Calcul des prix moyens

In [ ]:
atm.mean_gare(prix)

## Affichage du swarmplot

On essaie d'afficher le swarmplot de gares avec des moyennes très differentes (calculées ci-dessus).

In [ ]:
gare = ['MONTPELLIER', 'NAILLOUX', 'BRAM','NARBONNE SUD']
atm.swarm_gare_price(prix, gare)

## Recherche des trajets optimaux

In [ ]:
start = input(f"Donnez une gare de départ parmi les gares suivantes : {list(prix.index)}.")
target = input(f"Donnez une gare d'arrivée parmi les gares suivantes : {list(prix.index)}.")
n = input("Rentrez le nombre de sortie que vous vous accordez :")

g, opt = atm.kmin_cost_out(data_dist=dist, data_price=prix,
                          start = start, target = target, 
                          k = int(n))
print(f"Pour avoir le trajet optimal entre {g[0]} et {g[-1]},\n",
    f"il faut sortir à {g[1:-1]}. Le coût total sera de {round(opt, 3)}€")

## Une utilisation intelligente

On va chercher à afficher les sorties pour le trajet optimal sur une carte.

In [ ]:
import folium
from asltam.io.pricedist import get_index

start = 'SETE'
target = 'PERPIGNAN NORD'
m = atm.trajet(start, target, geo, prix, dist , 
               '5b3ce3597851110001cf62486f5564a064e34f3895221e5a0d9a2405')

g, opt = atm.kmin_cost_out(data_dist=dist, data_price=prix,
                          start = start, target = target, 
                          k = 4)
# intégrons ces résultats dans la carte m (les données étant compatibles).
for i in range(len(g)):
    k = get_index(geo, g[i])
    coor = list(geo.iloc[k])
    folium.Marker(coor,
    popup=g[i], 
    icon=folium.Icon(color='red', icon='car', prefix='fa')).add_to(m)

print(f"Coût du trajet : {opt}.")

In [ ]:
m